In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk

In [26]:
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')

In [50]:
from datetime import datetime
 
year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year
df_train['year'] = df_train['datetime'].map(year)
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
df_train['month'] = df_train['datetime'].map(month)
hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour
df_train['hour'] = df_train['datetime'].map(hour)
week_number = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime('%V')
df_train['week_number'] = df_train['datetime'].map(week_number)
day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
df_train['day_of_week'] = df_train['datetime'].map(day_of_week)

year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year
df_test['year'] = df_test['datetime'].map(year)
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
df_test['month'] = df_test['datetime'].map(month)
hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour
df_test['hour'] = df_test['datetime'].map(hour)
week_number = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime('%V')
df_test['week_number'] = df_test['datetime'].map(week_number)
day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
df_test['day_of_week'] = df_test['datetime'].map(day_of_week)

df_train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,week_number,day_of_week
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,0,52,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,52,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,2,52,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,3,52,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,4,52,5


In [53]:
day_part=[]
for i in df_train['hour']:
    if i>=0 and i<6:
        day_part.append(1)
    elif i>=6 and i<12:
        day_part.append(2)
    elif i>=12 and i<18:
        day_part.append(3)
    elif i>=18 and i<=23:
        day_part.append(4)
        
df_train['day_part']=pd.DataFrame(day_part)       

In [54]:
day_part=[]
for i in df_test['hour']:
    if i>=0 and i<6:
        day_part.append(1)
    elif i>=6 and i<12:
        day_part.append(2)
    elif i>=12 and i<18:
        day_part.append(3)
    elif i>=18 and i<=23:
        day_part.append(4)
        
df_test['day_part']=pd.DataFrame(day_part)


In [55]:
"""day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
week_number = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime('%V')"""

'day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()\nmonth = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month\nweek_number = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime(\'%V\')'

In [56]:
X_train = df_train[['day_part','week_number','day_of_week','hour','season','holiday','workingday','weather','temp','atemp','humidity','windspeed']]
y_train= df_train['count']
X_test_data= df_test[['day_part','week_number','day_of_week','hour','season','holiday','workingday','weather','temp','atemp','humidity','windspeed']]


In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
dt_rf = DecisionTreeClassifier(random_state=0)
regr_rf = RandomForestRegressor(n_estimators=200,max_depth=3,random_state=0)
clf_dtree=dt_rf.fit(X_train, y_train)
clf = regr_rf.fit(X_train, y_train)


In [58]:
df_test['predicted_count'] = [data for data in dt_rf.predict(X_test_data)]

In [59]:
writer = pd.ExcelWriter('predicted_data_dtree.xlsx')
df_test.to_excel(writer,'Sheet1')
writer.save()

In [60]:
import pickle;
# open a file, where you ant to store the data
file = open('important_dtree', 'wb')

# dump information to that file
pickle.dump(clf_dtree, file)

# close the file
file.close()

# open a file, where you stored the pickled data
file = open('important_dtree', 'rb')

# dump information to that file
clf2 = pickle.load(file)

# close the file
file.close()


In [61]:
#we can also use clf.predict for predicting data after loading model into pickel file
clf.predict(X_test_data)

array([ 44.5544531 ,  33.99507288,  16.37951814, ..., 123.72632248,
       123.72632248, 123.72632248])

In [63]:
#predicting using decision tree classifier
clf_dtree.predict(X_test_data)

array([13,  6,  4, ..., 97, 59, 41], dtype=int64)